In [1]:
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
import os

2024-01-22 11:25:17.893331: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 11:25:18.537360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import psutil
psutil.virtual_memory().total  # total physical memory in Bytes

134740668416

In [3]:
def get_image_paths(directory):
    image_paths = []
    valid_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more extensions if needed

    for filename in os.listdir(directory):
        if any(filename.lower().endswith(ext) for ext in valid_extensions):
            image_paths.append(os.path.join(directory, filename))

    return image_paths

In [6]:
def get_vector(img_path): 
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)

    return features.flatten()

In [20]:
import tensorflow as tf


# Load pre-trained VGG16 model
model = VGG19(weights='imagenet', include_top=True)

# Load and preprocess your dataset images
# Assuming 'image_paths' is a list of paths to your images
embeddings = []
image_paths = get_image_paths("/home/mahdi/FairnessLens/PreProcessor/colored/")
train_paths = image_paths[:int(0.8 * len(image_paths))]
test_paths = image_paths[int(0.8 * len(image_paths)):]
print(len(image_paths), len(train_paths), len(test_paths))
for img_path in train_paths:
    embeddings.append(get_vector(img_path))

574710816/574710816 [==============================] - 24s 0us/step
692 553 139
1/1 [==============================] - 0s 104ms/step


In [21]:
len(embeddings[1])

1000

In [22]:
mean_vector = np.mean(embeddings, axis=0)
mean_vector

array([6.73309376e-04, 5.35652252e-05, 3.75241543e-05, 3.37785968e-05,
       1.21034464e-04, 8.76619160e-05, 3.70702364e-05, 2.63768707e-05,
       1.45067515e-05, 1.33263238e-05, 2.98080522e-05, 2.54366023e-05,
       2.44968433e-05, 1.32937203e-05, 2.90577627e-05, 1.79823346e-05,
       5.77975770e-05, 2.27197215e-05, 1.96363708e-05, 2.98049763e-05,
       9.62086779e-06, 1.38819843e-04, 1.19023345e-04, 6.14332021e-05,
       3.36499797e-05, 1.48344543e-05, 1.34527909e-05, 3.63807994e-05,
       6.90531961e-05, 1.50785218e-05, 7.66518497e-06, 1.07368387e-05,
       1.75166624e-05, 1.50283413e-05, 2.67573487e-05, 3.08128210e-06,
       2.64316295e-05, 6.84425549e-06, 2.24320738e-05, 2.86863178e-05,
       9.23813877e-06, 7.57803355e-06, 6.91772357e-06, 4.23336351e-05,
       1.11584814e-05, 2.45888186e-05, 1.84986766e-05, 2.27854634e-05,
       6.00001658e-06, 6.04741717e-06, 4.10715220e-05, 1.24212820e-04,
       2.30419628e-05, 2.60609013e-05, 2.34368708e-05, 2.42458245e-05,
      

In [23]:
covariance_matrix = np.cov(embeddings, rowvar=False)
covariance_matrix

array([[3.52024348e-06, 2.97948937e-08, 4.73462584e-08, ...,
        3.51848232e-08, 2.56127764e-07, 6.94329684e-09],
       [2.97948937e-08, 4.13955043e-09, 1.03604075e-09, ...,
        9.98857731e-10, 1.61800525e-08, 1.73838529e-08],
       [4.73462584e-08, 1.03604075e-09, 2.70444486e-09, ...,
        8.54514459e-10, 6.80266593e-09, 1.09931434e-08],
       ...,
       [3.51848232e-08, 9.98857731e-10, 8.54514459e-10, ...,
        1.13771256e-09, 8.98966355e-09, 7.52844570e-09],
       [2.56127764e-07, 1.61800525e-08, 6.80266593e-09, ...,
        8.98966355e-09, 1.62150109e-07, 1.14235444e-07],
       [6.94329684e-09, 1.73838529e-08, 1.09931434e-08, ...,
        7.52844570e-09, 1.14235444e-07, 1.17469586e-06]])

In [24]:
covariance_matrix.shape

(1000, 1000)

In [25]:
covariance_matrix_regularized = covariance_matrix 
covariance_matrix_inverse = np.linalg.inv(covariance_matrix_regularized)

In [26]:
def get_score(image_path, mean_vec, cov_inverse):
    v = get_vector(image_path)
    diff = v - mean_vec
    res = np.dot(np.dot(diff.T, cov_inverse), diff)
    return res

In [30]:
scores = []
for img in test_paths: 
    scores.append(get_score(img, mean_vector, covariance_matrix_inverse))

1/1 [==============================] - 0s 92ms/step


In [31]:
sorted(scores)

[-1.2065009799920728e+17,
 -6.9817922950062264e+16,
 -3.1024013552554188e+16,
 -2.5655749615771268e+16,
 -2.2221792139147908e+16,
 -1.5593823689394444e+16,
 -1.4147827938688936e+16,
 -1.29665252928306e+16,
 -1.1066911752800648e+16,
 -1.0158230560691428e+16,
 -1.0051672121703774e+16,
 -9185259844229642.0,
 -9159508866330404.0,
 -8490440476966742.0,
 -7983526653470368.0,
 -7518511002653461.0,
 -6607450756319090.0,
 -4275547348341439.5,
 -4129270518372164.5,
 -3982088689568445.5,
 -3840550050972985.5,
 -3681512089466141.0,
 -3346922102847844.0,
 -3130130851677148.5,
 -2918712166199982.0,
 -2443576961646157.0,
 -2036530384530872.8,
 -1881199293635079.2,
 -1837283166176742.5,
 -1809284246744010.8,
 -1525133179481846.2,
 -1514340366471060.8,
 -1396589388135320.5,
 -1342001893034402.2,
 -1220194860732030.5,
 -1202360297486962.2,
 -908925585992467.0,
 -881508452614550.8,
 -517874833167627.44,
 -507147763857811.8,
 -505667069223655.9,
 -496397718162978.75,
 -479552689658492.7,
 -443342460768763

In [18]:
sorted(scores) # vgg19 scores

[4809288472.562568,
 5081274331.425193,
 6792965113.321077,
 7032839820.226066,
 7245742246.178535,
 8419788681.347321,
 8613131874.407621,
 8863732911.624706,
 9755833543.03695,
 9843279241.715464,
 10874964158.688198,
 11279543857.75792,
 11694296246.28775,
 12057233677.933695,
 12822914233.65768,
 13353702735.160315,
 13579431802.889156,
 13915600827.022774,
 14613855190.582397,
 14846371036.6277,
 15431429021.74649,
 15648083649.531507,
 16185909314.945063,
 16698107914.579123,
 17364911580.934944,
 17712230407.081318,
 17957460074.138443,
 17958797964.977234,
 18173627476.15654,
 18464097729.114246,
 18963963888.950073,
 19267493195.26809,
 19731829076.071056,
 20048602396.53046,
 20634136517.816143,
 21459998554.218224,
 21686644027.86033,
 21959623071.89093,
 23327373558.296825,
 24186863324.057358,
 24284116010.953648,
 24956765637.979042,
 25086600334.730583,
 25702452489.50263,
 26044169969.520058,
 26165475929.968807,
 26332652102.00106,
 26504674572.660748,
 26903016542.290

In [ ]:
s

In [31]:
sorted(scores) # vgg16 scores

[7613800226.915246,
 9883645641.829054,
 10507733301.186066,
 10820601743.948761,
 10872407949.554705,
 11172280169.44218,
 11610573883.98426,
 13377389069.235,
 13494420058.822527,
 13767190721.8024,
 13807547138.328661,
 14436864400.746593,
 16140731218.045092,
 16806755208.041218,
 16937276394.426142,
 18012673211.019836,
 19316940284.07759,
 19807956017.39718,
 21102933736.66299,
 21247848848.571278,
 21476880493.678864,
 21811531517.29067,
 22983083732.454933,
 23125440205.891327,
 23218522542.66873,
 24388188332.058086,
 24412443954.59689,
 24502606244.39663,
 25993408525.45211,
 26371308629.397484,
 26776755530.78212,
 27366012706.211353,
 27650318193.719025,
 27901601891.271515,
 27972405215.66484,
 28115267242.224457,
 28231260734.90865,
 28331504253.597805,
 28944420815.92607,
 29074514224.82479,
 29091656817.55135,
 30160506730.877834,
 30760436888.133938,
 32179136045.53276,
 32471551780.027763,
 33321256721.562454,
 33797826174.25977,
 35095599351.75374,
 35558119773.50787